In [15]:
import pandas as pd
from numpy import dot 
from numpy.linalg import norm

from transformers import AutoTokenizer

In [16]:
data = pd.read_csv('/Users/jhyun/Git/final-project-level3-nlp-16-1/data/raw_csv_data/600100001_data.csv')

In [17]:
data.head()

,Unnamed: 0,title,tag,location,description,img,image_count,pid,url
0,0,2022수능 수험생 이벤트 노트북 LG그램16,"노트북,중고노트북,게임노트북,그램,수험생할인",인천광역시 남동구 만수4동,2022 수능 수험생 여러분 고생하셨습니다\n2022년 수능 수험 표 인증하시면...,https://media.bunjang.co.kr/product/170628386_...,6,170628386,https://m.bunjang.co.kr/products/170628386
1,1,M1 맥북에어 CTO 16GB 256GB 골드 풀박스,"맥북에어,m1맥북에어,m1,맥북",서울특별시 종로구 혜화동,M1 맥북에어 CTO 16GB 256GB 골드 풀박스\n\n사용을 많이 안해서 상태...,https://media.bunjang.co.kr/product/170714207_...,12,170714207,https://m.bunjang.co.kr/products/170714207
2,2,델i7노트북팝니다,"델노트북,i7,게임용",인천광역시 남동구 만수5동,내용은 사진참조요,https://media.bunjang.co.kr/product/170714168_...,12,170714168,https://m.bunjang.co.kr/products/170714168
3,3,[미개봉] LG전자 노트북 2021 그램16 16Z90P-GP5EL,"노트북,중고노트북,그램,gram,미개봉노트북",인천광역시 남동구 만수4동,★ 번개패이 이용해서 카드결제 가능합니다~~~^^\n★ 다량 입고 되어서 저렴하게 ...,https://media.bunjang.co.kr/product/166453553_...,11,166453553,https://m.bunjang.co.kr/products/166453553
4,4,LG A515_i5 2410m 노트북 판매합니다.,노트북,서울특별시 강서구 화곡동,LG A515_i5 2410m 노트북 판매합니다.\n\n윈도우는 새로 설치했습니다....,https://media.bunjang.co.kr/product/170714073_...,6,170714073,https://m.bunjang.co.kr/products/170714073


### 1. 전체 데이터에 대해서 hashtag vocab 만들기 
### 2. title + description concat 연결 -> 2500개 
### 3. 1번, 2번에 대해서 유사도 구하고 hashtag vocab에서 해시태그 뽑기 

In [18]:
data

,Unnamed: 0,title,tag,location,description,img,image_count,pid,url
0,0,2022수능 수험생 이벤트 노트북 LG그램16,"노트북,중고노트북,게임노트북,그램,수험생할인",인천광역시 남동구 만수4동,2022 수능 수험생 여러분 고생하셨습니다\n2022년 수능 수험 표 인증하시면...,https://media.bunjang.co.kr/product/170628386_...,6,170628386,https://m.bunjang.co.kr/products/170628386
1,1,M1 맥북에어 CTO 16GB 256GB 골드 풀박스,"맥북에어,m1맥북에어,m1,맥북",서울특별시 종로구 혜화동,M1 맥북에어 CTO 16GB 256GB 골드 풀박스\n\n사용을 많이 안해서 상태...,https://media.bunjang.co.kr/product/170714207_...,12,170714207,https://m.bunjang.co.kr/products/170714207
2,2,델i7노트북팝니다,"델노트북,i7,게임용",인천광역시 남동구 만수5동,내용은 사진참조요,https://media.bunjang.co.kr/product/170714168_...,12,170714168,https://m.bunjang.co.kr/products/170714168
3,3,[미개봉] LG전자 노트북 2021 그램16 16Z90P-GP5EL,"노트북,중고노트북,그램,gram,미개봉노트북",인천광역시 남동구 만수4동,★ 번개패이 이용해서 카드결제 가능합니다~~~^^\n★ 다량 입고 되어서 저렴하게 ...,https://media.bunjang.co.kr/product/166453553_...,11,166453553,https://m.bunjang.co.kr/products/166453553
4,4,LG A515_i5 2410m 노트북 판매합니다.,노트북,서울특별시 강서구 화곡동,LG A515_i5 2410m 노트북 판매합니다.\n\n윈도우는 새로 설치했습니다....,https://media.bunjang.co.kr/product/170714073_...,6,170714073,https://m.bunjang.co.kr/products/170714073
...,...,...,...,...,...,...,...,...,...
2495,2495,갤럭시북 플렉스 알파 NT750QCJ[가격내림],"삼성노트북,갤럭시북플렉스알파,15인치,NT750QCJ",경기도 수원시 권선구 권선2동,삼성노트북플렉스알파 NT750QCJ 램8기가 SSD512 15인치 판매합니다\n\n...,https://media.bunjang.co.kr/product/170145708_...,4,170145708,https://m.bunjang.co.kr/products/170145708
2496,2496,ssd하드 노트북 마우스 파우치 드려요,"노트북,삼성,마우스",서울특별시 강동구 암사제1동,사양은 사진 보시면 나옵니다.\n2014년 것이고요.\n\n정품은 3년전까지 70만...,https://media.bunjang.co.kr/product/143817706_...,8,143817706,https://m.bunjang.co.kr/products/143817706
2497,2497,노트북 전자기기 최고가 매입합니다 삽니다,"노트북매입,노트북삽니다,노트북출장,중고매입,노트북",인천광역시 연수구 송도2동,■■번개톡확인잘안합니다 (꼭!!) 밑에번호나 카카오플러스로문의주세요^^)\r\n\r...,https://media.bunjang.co.kr/product/166265753_...,1,166265753,https://m.bunjang.co.kr/products/166265753
2498,2498,에이서 S7-391 i5 부품용 터치 노트북,"노트북부품,부용용노트북,노트북부품용",경상남도 창원시 성산구 성주동,Acer Aspire S7-391-533314G12aws 입니다.\n터치 디스플레이...,https://media.bunjang.co.kr/product/162293546_...,2,162293546,https://m.bunjang.co.kr/products/162293546


In [19]:
# 1. 전체 데이터에 대해서 hashtag vocab
hashtag_vocab = []
for tag in list(data['tag']):
    hashtag_vocab.extend(tag.split(','))


hashtag_vocab = list(set(hashtag_vocab))
hashtag_vocab # 2118 

['hpenvy',
 '맥북프로논터치바',
 'GIGABYTE',
 '삼성x11노트북',
 '맥북프로2016터치바',
 '슈퍼플라이',
 '고해상도노트북',
 '밀양',
 '갤럭시북프로',
 'xps',
 '택배가능',
 '삼성오토캐드노트북',
 '레노버탭48플러스',
 '삼성램',
 '강서구양천구발산동',
 '문서용',
 '2020맥북프로13인치',
 '보험',
 '램4',
 'NT905S3G',
 'hp스트림',
 '맥북키캡',
 '르누아르노트북',
 '홈런',
 '128',
 '대구중고컴퓨터',
 'i5',
 '노트북캐드',
 '포토샵용노트북',
 '모니터',
 '15Z990',
 '노트북렘',
 '#맥북프로15인치',
 '스마트폰매입',
 '광주',
 '부천시김포시고양시',
 '에이수스노트북',
 '제네시스키케이스',
 'zephyrus',
 'LG노트북가방',
 '맥북에어2012',
 '서피스프로3',
 '12인치노트북학생용노트북',
 '14',
 '갤럭이온',
 '초경량',
 '맥북256',
 '맥북2019',
 '외장하드500기가',
 '플립북',
 'lg울트라pc',
 '엔비디아',
 '노트북중고',
 '스트릭스',
 '유튜브',
 '과제용',
 '에일리언웨어노트북',
 '정품가죽파우치',
 '250기가',
 '아티브9',
 'GTX910M',
 '매입',
 '노트북인강용',
 '8gb',
 '아이패드필름',
 'lg그램13인치',
 '삼성크롬북',
 '4K',
 '로직프로',
 'asus박스',
 '13인치',
 '2060',
 'B560M',
 '인텔i7',
 '삼성노트북7',
 '삼성노트북플러스2',
 '한성아방가르드',
 '1660',
 '엘지',
 '구매',
 'i5-2430M',
 '맥북프로17인치',
 '제네시스차키',
 '노트북고장',
 '그래픽',
 '2in1플립북',
 '인스피론',
 '매북프로',
 '미사용',
 '엘라고스탠드',
 '최저가노트북',
 '맥북미개봉',
 '17Z90P',
 '맥미니m1',
 '충주',
 '서피스

In [20]:
MODEL_NAME = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [21]:
# tokenizer(hashtag_vocab)

In [22]:
hashtag_vector = []
hashtag_token = []
for id in tokenizer(hashtag_vocab)['input_ids']:
    id = id[1:-1]
    hashtag_vector.append(id)
    token = tokenizer.convert_ids_to_tokens(id)
    hashtag_token.append(token)
hashtag_vector[:3], hashtag_token[:3]

([[75, 2006, 3988, 2045, 2076],
  [1051, 2547, 17287, 2473, 2115, 2225, 2100],
  [43, 2184, 12170, 2206, 2214, 7832]],
 [['h', '##p', '##en', '##v', '##y'],
  ['맥', '##북', '##프로', '##논', '##터', '##치', '##바'],
  ['G', '##I', '##GA', '##B', '##Y', '##TE']])

In [23]:
# 2. title + description concat

In [24]:
text_concat = []
for i in range(len(data)):
    row = data.iloc[i]
    text = row['title'] + ' ' + row['description']
    text = text.replace('\n', ' ')
    text_concat.append(text)

In [25]:
text_concat_vector = []
text_concat_token = []
for id in tokenizer(text_concat)['input_ids']:
    id = id[1:-1]
    text_concat_vector.append(id)
    token = tokenizer.convert_ids_to_tokens(id)
    text_concat_token.append(token)

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


In [26]:
# 3. 유사도 구하기 

In [27]:
def make_matrix(feats, list_data):
    freq_list = []
    for feat in feats:
        freq = 0 
        for word in list_data: 
            if feat == word: 
                freq += 1 
        freq_list.append(freq) 
    return freq_list

In [28]:
result = []
for vocab_word in hashtag_token:
    sim = make_matrix(vocab_word, text_concat_token[0])
    word = tokenizer.convert_tokens_to_string(vocab_word)
    result.append([word, sum(sim)/len(sim)])

In [29]:
print(tokenizer.convert_tokens_to_string(text_concat_token[0]))

2022수능 수험생 이벤트 노트북 LG그램16 2022 수능 수험생 여러분 고생하셨습니다 2022년 수능 수험 표 인증하시면 그램16 구입하시는 수험생에겐 ▶ SSD 256GB 무료추가 ▶ 5만원 할인 둘중 1가지 선택 해서 적용해 드립니다 ◆ 적용제품 ( 제품 세부 사양은 판매상품 참조 ) ▶ 16Z90P - GP50ML ( i5 8GB 256GB ) ▶ 16Z90P - GP5EL ( i5 16GB 256GB ) ▶ 16Z90P - GP70ML ( i7 16GB 256GB ) 전제품 윈도우10정품 내장, 무상서비스1년


In [30]:
result_sort = sorted(result, key = lambda x : x[1], reverse=True)

In [31]:
result_sort[:10]

[['16GB', 6.333333333333333],
 ['256GB', 6.0],
 ['8GB', 5.0],
 ['16', 5.0],
 ['256', 4.0],
 ['램16GB', 4.0],
 ['16Gb', 4.0],
 ['8G', 4.0],
 ['ROG', 3.5],
 ['4G', 3.5]]

In [32]:
result_sort[-10:]

[['게이밍기어', 0.0],
 ['애플펜슬2세대', 0.0],
 ['인천', 0.0],
 ['dell', 0.0],
 ['usb허브', 0.0],
 ['요가', 0.0],
 ['3050ti', 0.0],
 ['맥북에어m1512', 0.0],
 ['맥북프로2014', 0.0],
 ['lg노트북', 0.0]]